In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [11]:

import pandas as pd
import numpy as np
import torch
# import torchio as tio
# from matplotlib import pyplot as plt
from pathlib import Path
from tqdm.autonotebook import tqdm
# import zarr
# from nibabel import ecat
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

from data_loading.data_set import EcatDFDataset, get_default_tio_transform
from torch.utils.data import DataLoader
from deep_learning.Lit3dResnet import Lit3dResnet

In [3]:
path_df = Path('/home/fabian/projects/phd/ai_radiation_therapy/ai_head_neck_tumor/data/clinical_data_has_CT_PT_21feb23.feather')
df = pd.read_feather(path_df)
data_path = Path(df.iloc[0].blue_data_path)
ds_train = EcatDFDataset(df, mode='train', cv_split=0,  img_columns=['CT_img_path', 'CT_roi_path', 'PT_img_path', 'PT_roi_path']
                         , transform=get_default_tio_transform('train'))
ds_valid = EcatDFDataset(df, mode='valid', cv_split=0,  img_columns=['CT_img_path', 'CT_roi_path', 'PT_img_path', 'PT_roi_path']
                         , transform=get_default_tio_transform('valid'))

In [4]:
dl_train = DataLoader(ds_train, batch_size=8, shuffle=True, num_workers=8)
dl_valid = DataLoader(ds_valid, batch_size=8, shuffle=False, num_workers=8)

In [38]:
import socket
print(socket.gethostname() == 'blue')

True


In [5]:
batch_x, batch_y = next(iter(dl_train))
print(batch_y, batch_x.shape)

tensor([1, 0, 0, 1, 0, 1, 1, 1]) torch.Size([8, 4, 128, 128, 140])


In [12]:
optimizer = opt_class(model.parameters(), lr=1e-4)

In [7]:
# criterion = nn.CrossEntropyLoss()

# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
# # optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [29]:
columns = ['CT_img_path', 'CT_roi_path', 'PT_img_path', 'PT_roi_path']
selector = 6
indices = [[0], [2], [0, 1], [2, 3], [0, 1, 2, 3], [0, 2], [0, 2, 3]]
selected_columns = [columns[i] for i in indices[selector]]
info_cols = ''.join(selected_columns)
info_cols

'CT_img_pathPT_img_pathPT_roi_path'

## Train in pytorch, as pytorch-lightning seems to not work for now..

In [23]:

def validate(model, testloader, criterion, device='cuda', disable_tqdm=False):
    model.eval()
    print('Validation')
    valid_losses = []
    valid_accs = []
    valid_preds = []
    valid_targets = []
    valid_running_correct = 0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(testloader), total=len(testloader), disable=disable_tqdm):
            counter += 1
            
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = criterion(outputs, labels)
            valid_losses.append(loss.item())
            # Calculate the accuracy.
            _, preds = torch.max(outputs.data, 1)
            valid_accs.append((preds == labels).float().mean().item())
            valid_preds.append(preds)
            valid_targets.append(labels)
        
    # Loss and accuracy for the complete epoch.
    model.train()
    valid_preds = torch.concat(valid_preds).cpu().numpy()
    valid_targets = torch.concat(valid_targets).cpu().numpy()
    valid_accs = np.array(valid_accs)
    valid_losses = np.array(valid_losses)
    return valid_losses, valid_accs, valid_preds, valid_targets

In [8]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
non_trainable_params = sum(p.numel() for p in model.parameters() if not p.requires_grad)

print(f"Number of trainable parameters: {trainable_params}")
print(f"Number of non-trainable parameters: {non_trainable_params}")

Number of trainable parameters: 46159170
Number of non-trainable parameters: 0


In [24]:

model.train()
device = 'cuda'
model.cuda()
loss_acc_dict = {"train_loss": [], "valid_loss": [], "train_acc": [], "valid_acc": [], "epoch": [], 'valid_preds': [],
                 'valid_targets': []}
epochs = 1

for epoch in range(epochs):
    model.train()
    epoch_train_losses = []
    epoch_train_accs = []
    pbar = tqdm(total=len(dl_train))
    for i, data in enumerate(dl_train):
            # counter += 1
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = criterion(outputs, labels)
            epoch_train_losses.append(loss.item())
            # Calculate the accuracy.
            _, preds = torch.max(outputs.data, 1)
            epoch_train_accs.append((preds == labels).sum().item())
            # Backpropagation
            loss.backward()
            # Update the weights.
            optimizer.step()
            # set the progress bar
            pbar.set_description(f"Epoch {epoch+1}/{epochs}, loss: {loss.item():.4f}")
            pbar.update(1)
    # Loss and accuracy for the complete epoch.
    loss_acc_dict['train_loss'].append(np.mean(epoch_train_losses))
    loss_acc_dict['train_acc'].append(np.mean(epoch_train_accs))
    
    valid_losses, valid_accs, valid_preds, valid_targets = validate(model, dl_valid, criterion, device='cuda', disable_tqdm=False)
    valid_accuracy = (valid_preds == valid_targets).mean()
    print(f"Validation loss: {np.mean(valid_losses):.4f}, accuracy: {valid_accuracy:.2%}")
    loss_acc_dict['valid_loss'].append(np.mean(valid_losses))
    loss_acc_dict['valid_acc'].append(valid_accuracy)
    loss_acc_dict['valid_preds'].append(valid_preds)
    loss_acc_dict['valid_targets'].append(valid_targets)

  0%|          | 0/104 [00:00<?, ?it/s]

Validation


  0%|          | 0/26 [00:00<?, ?it/s]

Validation loss: 0.7185, accuracy: 48.08%


# Create Pytorch-lightning module

In [19]:
params = {'lr': 0.001, 'adam_regularization': 0.01, 'lr_decay': 0.95, 'resnet_model_size': 50, 'model_chan_in': 4, 'n_classes': 2}
model = Lit3dResnet(params)

trainer = pl.Trainer(gpus=1, max_epochs=1, enable_checkpointing=False,
                     limit_train_batches=0.1, limit_val_batches=0.1)
trainer.fit(model, dl_train, dl_valid)
# trainer.save_checkpoint("example.ckpt")


/home/fabian/anaconda3/envs/torchenv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:474: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 46.2 M
------------------------

Sanity Checking: 0it [00:00, ?it/s]

/home/fabian/anaconda3/envs/torchenv/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1609: PossibleUserWarning: The number of training batches (10) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 1/1 complete.
Training loss: 1.1291; Training accuracy: 51.25%
Validation loss: 86.5851; Validation accuracy: 87.50%


In [28]:
import pickle

In [33]:
path_checkpoints = Path('/home/fabian/projects/phd/ai_radiation_therapy/ai_head_neck_tumor/data/checkpoints')
params['loss_acc_dict'] = model.loss_acc_dict
trainer.save_checkpoint(path_checkpoints / "test.ckpt")
pickle.dump(params, open(path_checkpoints / "test_params.pkl", "wb"))

In [34]:
test = Lit3dResnet.load_from_checkpoint(path_checkpoints / "test.ckpt", params=params)


In [35]:
model.loss_acc_dict

{'train_loss': [0.8166417427934133,
  0.7375371315731452,
  0.7277085984555575,
  0.7163092359327353,
  0.7064844593405724,
  0.6972267272380682,
  0.7002557332699115,
  0.688205395753567,
  0.6957612321353875,
  0.6932428006369334],
 'valid_loss': [tensor(82.0294, device='cuda:0'),
  tensor(3.3975, device='cuda:0'),
  tensor(0.6862, device='cuda:0'),
  tensor(0.6738, device='cuda:0'),
  tensor(5.9927, device='cuda:0'),
  tensor(0.8164, device='cuda:0'),
  tensor(0.9275, device='cuda:0'),
  tensor(0.7250, device='cuda:0'),
  tensor(0.6833, device='cuda:0'),
  tensor(0.7295, device='cuda:0'),
  tensor(0.7549, device='cuda:0')],
 'train_acc': [0.48851269649334944,
  0.5030229746070133,
  0.5102781136638452,
  0.5042321644498187,
  0.5332527206771464,
  0.5259975816203144,
  0.5259975816203144,
  0.5695284159613059,
  0.5646916565900847,
  0.5501813784764208],
 'valid_acc': [0.875,
  0.5192307692307693,
  0.5432692307692307,
  0.5769230769230769,
  0.4807692307692308,
  0.5673076923076923